# Model Description

This notebook describes the creation of the RR Lyrae model. For usage examples see the `demo_R.ipynb` and `demo_python.ipynb` notebooks.

## Model Form

The form for the RR Lyrae model is
$$ m_b(t) = \mu + M_b(\omega) + E[B-V]R_b + a \gamma_b(\omega t + \phi)$$
where the population parameters, common to all RR Lyrae, are:
\begin{align*}
  M_b(\omega) &= \text{ ab. magnitude in band b for an RR Lyrae with frequency } \omega\\
  R_b &= \text{extinction law in band b}\\
  \gamma_b &= \text{the shape of the RR Lyrae light curve in band b}
\end{align*}

The object specific parameters, different for each RR Lyrae, are
\begin{align*}
  \mu &= \text{distance modulus}\\
  E[B-V] &= \text{amount of dust}\\
  a &= \text{amplitude}\\
  \omega &= \text{frequency}\\
  \phi &= \text{ phase}
\end{align*}

For one RR Lyrae we observe photometry $\{\{t_{bi},m_{bi},\sigma_{bi}\}_{i=1}^{n_b}\}_{b=1}^B$ where $m_{bi}$ is the observed magnitude at time $t_{bi}$ in filter $b$ measured with (known) uncertainty $\sigma_{bi}$. The bands are indexed $1,\ldots,B$ instead of typical letters e.g. ugriz. The model and data are related by
$$m_{bi} = m_b(t_{bi}) + \epsilon_{bi}$$
where $\epsilon_{bi} \sim N(0,\sigma_{bi}^2)$. More generally we could view $\epsilon_{bi}$ as a mean $0$ random variable with $Var(\epsilon_{bi}) = \sigma_{bi}^2$. We discuss these modeling asssumptions in more detail in the Estimating Object Parameters section of this note.

This model assumes all RR Lyrae share a common shape by band and that RR Lyrae are strictly periodic functions. These assumptions are an approximation. For example, RR Lyrae which undergo the Blazhko effect are not strictly periodic functions. Rather than construct a perfectly accurate model, the goal is to construct a model with few free parameters that provides a better approximation to RR Lyrae variation than existing methods. For example a simple sinusoid model fit to 5 filters has a total of 16 free parameters (5 means, 5 amplitude, 5 phases, and 1 frequency) while providing only a very rough approximation to the steep rise and slow decline in brighness observed in RR Lyrae light curves. In contrast this model provides a significantly better approximation while fitting for 5 free parameters (or 4 if light curves are dust corrected). We provide empirical evidence to support these claims in our results section. 

## Estimating Population Parameters

We estimate the population parameters from theory and existing data sets. The file `code/make_template/make_template.R` in the rr-model repository contains code for doing this. Here we describe the process.



Perhaps state the $R_b$, $M_b(\omega)$ we use, without describing the process.

Smooth light curves, phase align, mean 0, then solve for A and gamma.

Discuss estimating model error. <strong data-cite="long2017">(Long, 2017)</strong>


## Estimating Object Parameters

### References

Long 2017. "A note on parameter estimation for misspecified regression models with heteroskedastic errors." Electronic Journal of Statistics.